### 1. 환경 준비 & Pinecone 연결

In [ ]:
from dotenv import load_dotenv
import os
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.schema.runnable import RunnableLambda

load_dotenv()

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

#벡터스토어를 이용한 임베딩&적재  
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

vector_store = PineconeVectorStore(
    index_name=os.getenv("wine-reviews"),
    embedding=embedding,
    pinecone_api_key=os.getenv("PINECONE_API_KEY")
)


### 1-1. 기존에 생성한 인덱스가 있을 시

In [70]:
from pinecone import Pinecone
import os

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index("wine-reviews")   
stats = index.describe_index_stats()
print(stats)

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'wine_reviews_ns1': {'vector_count': 129971}},
 'total_vector_count': 129971,
 'vector_type': 'dense'}


### 2.와인 검색 함수

In [71]:
def search_wine(dish_flavor: str):
    results = vector_store.similarity_search(
        dish_flavor,
        k=5,
        namespace=os.getenv("PINECONE_NAMESPACE")
    )
    return {
        "dish_flavor": dish_flavor,
        "wine_reviews": "\n".join([doc.page_content for doc in results])
    }

### 3. 와인 추천 함수(from_messages 버전)

In [72]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def recommend_wine_chain(data):
    chat_template = ChatPromptTemplate.from_messages(
        [
            ("system", "당신은 전문 소믈리에입니다. 음식 풍미와 와인 리뷰를 바탕으로 최적의 와인을 추천하세요."),
            ("human", [
                {"type": "text", "text": f"음식 풍미 설명: {data['dish_flavor']}"},
                {"type": "text", "text": f"참고할 와인 리뷰:\n{data['wine_reviews']}"},
                {"type": "text", "text": "위 정보를 바탕으로 추천 와인과 이유를 알려주세요."}
            ])
        ]
    )
    chain = chat_template | llm | StrOutputParser()
    return chain.invoke({})


### 4.파이프라인 연결

In [75]:
# 22.ipynb에서 describe_dish_flavor_chain을 불러와야 함
# 방법 1: 함수 코드 복사
# 방법 2: %run "./22.와인이미지_랭체인프롬프트.ipynb"

runnable_1 = RunnableLambda(describe_dish_flavor_chain2)
runnable_2 = RunnableLambda(search_wine)
runnable_3 = RunnableLambda(recommend_wine_chain)

chain = runnable_1 | runnable_2 | runnable_3

response = chain.invoke({"image_urls": ["https://images.vivino.com/thumbs/Z90I3--JRKWlpMA8wdLY-Q_pb_x600.png"]})
print(response)


NameError: name 'describe_dish_flavor_chain2' is not defined